In [23]:
#The libraries needed to run the code and camb.
import math
import sys, platform, os
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import camb

#String Data
l, yt, zt, tt = np.loadtxt('cl_tt.d', unpack = True)
l, yte, zte, tte = np.loadtxt('cl_te.d', unpack = True)
l, yb, zb = np.loadtxt('cl_bb.d', unpack = True)
l, ye, ze, te = np.loadtxt('cl_ee.d', unpack = True)

#Noise curves for 2019-2020 servay from Srini
d = np.load('spt3g_winter_2020_ilc_cmb_90-150-220_TT-EE.npy', allow_pickle=True)
dd = d.item()


# Dictionaries for Cosmological Parameters.
# Each Dictionary is named after the variable we are varrying by 1 percent.
cos_dict = {'H0': 67.5, 'ombh2':0.022, 'omch2':0.122,'mnu': 0.06,'omk': 0 , 'tau': 0.06}
H0_dict ={'H0': 67.5*1.01, 'ombh2':0.022, 'omch2':0.122, 'mnu': 0.06,'omk': 0, 'tau': 0.06}
ombh2_dict = {'H0': 67.5, 'ombh2':0.022*1.01, 'omch2':0.122,'mnu': 0.06,'omk': 0, 'tau': 0.06}
omch2_dict={'H0': 67.5, 'ombh2':0.022, 'omch2':0.122*1.01, 'mnu': 0.06,'omk': 0, 'tau': 0.06}
tau_dict = {'H0': 67.5, 'ombh2':0.022, 'omch2':0.122, 'mnu': 0.06,'omk': 0, 'tau': 0.06*1.01}

    
def get_cl(dict,Gmu, b, c):
        """ The function get_cl finds the covariant matrix. 
    
        Parameters: 
        dict(dictinary): is the dictionary of cosmological parameters. 
        b(int): is the factor As is varied by. 
        c(int): is the factor ns is varied by. 
        
        Returns: 
        np.matrix: the matrix represents the covarient matrix. 
        """ 
        pars = camb.CAMBparams()
        cl_matrix = []
        pars.set_cosmology(**dict)
        pars.InitPower.set_params(As=b*2.10058296e-9, ns=c*0.96605, r =0)
        pars.set_for_lmax(10000, lens_potential_accuracy=0);
        results = camb.get_results(pars)
        powers =results.get_cmb_power_spectra(pars, CMB_unit='muK')
        totCL=powers['total']
        unlensedCL=powers['unlensed_scalar']
        ls = np.arange(totCL.shape[0])
        unlensedCL=powers['unlensed_scalar']
        cl_t = totCL[:,0]
        cl_e = totCL[:,1]
        cl_b = totCL[:,2]
        cl_te = totCL[:,3]
        noiseTT = dd['cl_residual']['TT']
        noiseEE = dd['cl_residual']['EE']
        SigmaTot_tt2 = 0
        SigmaStr_tt2 = 0
        for i in range(2,len(yt)):
            SigmaTot_tt2 += ((2*i+1)/(4*np.pi))* ((cl_t[i] *(2*np.pi))/(i*(i+1)))
            SigmaStr_tt2 += ((2*i+1)/(4*np.pi))* ( (yt[i]+ zt[i]+tt[i])*((2*np.pi))/(i*(i+1)))
        A = Gmu**2 * SigmaTot_tt2 *(1.27*10**(-6))**(-2) *(SigmaStr_tt2)**(-1)
        for i in ls[(ls < len(noiseTT)) & (ls < len(cl_t)) & (ls < len(yt))& (ls > 100)]:
            Cl_11 = A*(yt[i]+ zt[i]+tt[i])*(2*np.pi)/(i*(i+1))+noiseTT[i] + ((cl_t[i]*(2*np.pi))/(i*(i+1)))
            Cl_12 = A*(yte[i]+ zte[i]+te[i])*(2*np.pi)/(i*(i+1))+((cl_te[i]*(2*np.pi))/(i*(i+1)))
            Cl_22 = A*(ye[i]+ ze[i]+te[i])*(2*np.pi)/(i*(i+1))+noiseEE[i] + ((cl_e[i]*(2*np.pi))/(i*(i+1)))
            Cl_33 = ((cl_b[i]*(2*np.pi))/(i*(i+1))) +noiseEE[i]+ A*(yb[i]+ zb[i])*(2*np.pi)/(i*(i+1))
            c_l   = np.matrix([[Cl_11,  Cl_12, 0],[ Cl_12, Cl_22, 0], [0,0,Cl_33 ]])
            cl_matrix.insert(len(cl_matrix)-1,c_l)
        return cl_matrix

      
H0_cls = get_cl(H0_dict, 1*10**(-7),1, 1)
cos_cls = get_cl(cos_dict, 1*10**(-7),1,1)
Gmu_cls = get_cl(cos_dict, 1*10**(-7)*1.01,1,1)
ombh2_cls = get_cl(ombh2_dict, 1*10**(-7),1, 1)
omch2_cls = get_cl(omch2_dict,1*10**(-7), 1,1)
As_cls = get_cl(cos_dict,1*10**(-7),1.01,1)
ns_cls = get_cl(cos_dict, 1*10**(-7),1,1.01)
tau_cls = get_cl(tau_dict,1*10**(-7),1,1)
cl_dict = {"H0": H0_cls, "ombh2": ombh2_cls ,"omch2": omch2_cls, "As":As_cls, "ns":ns_cls, "tau":tau_cls, "Gmu":Gmu_cls, "cos": cos_cls}  
cl_dict_values = {"H0": 67.5, "ombh2": 0.022  ,"omch2": 0.122, "As":2.10058296e-9, "ns":0.96605, "tau":0.06, "Gmu":1*10**(-7)} 
var = list(cl_dict.keys())
f_matrix1 = np.zeros((7,7))
for row in range(7):
    for column in range(7):
        for i in range(1,2898):
                Clinv = np.linalg.inv((cl_dict[var[7]])[i])
                dCldtheta_row = ((((cl_dict[var[row]])[i]) - ((cl_dict[var[7]])[i])) / ((0.01)*cl_dict_values[var[row]]))
                dCldtheta_column = ((((cl_dict[var[column]])[i]) - ((cl_dict[var[7]])[i])) / ((0.01)*cl_dict_values[var[column]]))
                fsky = 1500 / 41253
                f_matrix1[row,column] += ((2*i+1)/2) * fsky * np.trace(Clinv * dCldtheta_row * Clinv * dCldtheta_column)
    
print(f_matrix1)

[[ 1.05697565e+02 -1.23567115e+05  3.75616513e+04 -9.60718312e+11
  -2.20759446e+03  3.70689786e+03 -7.49439083e+08]
 [-1.23567115e+05  2.50862225e+08 -4.22352349e+07  2.11273874e+15
   5.59651641e+06 -8.28386714e+06  1.73605713e+12]
 [ 3.75616513e+04 -4.22352349e+07  1.40913088e+07 -3.63363552e+14
  -7.70053103e+05  1.46756396e+06 -1.85296375e+11]
 [-9.60718312e+11  2.11273874e+15 -3.63363552e+14  5.19786044e+22
   9.55112684e+13 -2.00602905e+14  3.49727971e+19]
 [-2.20759446e+03  5.59651641e+06 -7.70053103e+05  9.55112684e+13
   2.18640581e+05 -3.73944689e+05  6.67438560e+10]
 [ 3.70689786e+03 -8.28386714e+06  1.46756396e+06 -2.00602905e+14
  -3.73944689e+05  7.83922048e+05 -1.24321853e+11]
 [-7.49439083e+08  1.73605713e+12 -1.85296375e+11  3.49727971e+19
   6.67438560e+10 -1.24321853e+11  4.87646604e+16]]


In [12]:
def prior(row,column, value, matrix):
    """The function adds a value to the defined value of the matrix . 
    
        Parameters: 
        row(int): specfies the row of the matrix.
        column(int): specifies the column of the matrix.
        value(int): the value added to the element specified by the row and column.
        matrix(np matrix): is the cl_matrix from get_cls
        
        Returns: 
        np.matrix: returns the covarient matrix with the updated value of the specified row and column."""
    matrix[row,column] +=   (1/value**2)
    return matrix
        

def error(matrix):    
    """The function adds a value to the defined value of the matrix . 
    
        Parameters: 
        matrix(np matrix): is updated matrix after adding a prior.
        
        Returns: 
        np.matrix: the uncertainities constraints on all the cosmological parameters."""

    a = np.linalg.inv(matrix)
    a.diagonal()
    for i in range(0, len(a.diagonal())): 
        errors = {}
        errors[i] = (a.diagonal()[i])**(1/2)
        print(errors)

In [13]:
prior(5,5,0.0070, f_matrix1)


array([[ 1.06272714e+02, -1.24476039e+05,  3.77047527e+04,
        -9.64891747e+11, -2.24947942e+03,  3.72303334e+03,
        -5.19364586e+08],
       [-1.24476039e+05,  2.52772173e+08, -4.24064349e+07,
         2.11949783e+15,  5.69632826e+06, -8.31104692e+06,
         1.14050673e+12],
       [ 3.77047527e+04, -4.24064349e+07,  1.41304264e+07,
        -3.63188203e+14, -7.78427418e+05,  1.46736526e+06,
        -1.19734001e+11],
       [-9.64891747e+11,  2.11949783e+15, -3.63188203e+14,
         5.19025842e+22,  9.64515302e+13, -2.00353357e+14,
         2.63791332e+19],
       [-2.24947942e+03,  5.69632826e+06, -7.78427418e+05,
         9.64515302e+13,  2.23685554e+05, -3.77502485e+05,
         4.70483575e+10],
       [ 3.72303334e+03, -8.31104692e+06,  1.46736526e+06,
        -2.00353357e+14, -3.77502485e+05,  8.03557904e+05,
        -9.25378270e+10],
       [-5.19364586e+08,  1.14050673e+12, -1.19734001e+11,
         2.63791332e+19,  4.70483575e+10, -9.25378270e+10,
         3.3875479

In [14]:
#cl_dict_values = {"H0": 67.5, "ombh2": 0.022  ,"omch2": 0.122, "As":2.10058296e-9, "ns":0.96605, "tau":0.06, "Gmu":1*10**(-7)}
error(f_matrix1)

{0: 0.6690361912465989}
{1: 0.0001534604993787167}
{2: 0.0017914198280124213}
{3: 2.7331805669016893e-11}
{4: 0.007482704251216383}
{5: 0.006817602684719896}
{6: 9.446985390342443e-09}


In [17]:
#finding the error in log(As).
2.7331805669016893e-11/ (np.log(10) * 2.10058296e-9)

0.005650837224018065